In [ ]:
import os
from os.path import isfile, join
import pandas as pd
outpath=r"C:\\Umass spring 20\\645\\github\\MLbasedDBMStuning\\Data\\Workload\\WorkloadB\\"

In [ ]:
## Functions to return file name in directory
def loadWorkloadFileNames(folder):
    onlyfiles = [f for f in os.listdir(folder) if isfile(join(folder, f))]
    return onlyfiles

In [ ]:
##Read workload
df_B=pd.read_csv('./Data/online_workload_B.csv')

In [ ]:
## Convert boolean knobs and workload id to integer if not already done
df_B['k7'] = df_B['k7'].apply(lambda x: 1 if x ==True else 0)
df_B['k7'] = df_B['k7'].apply(lambda x: 1 if x ==True else 0)
df_B['workload id']=df_B['workload id'].str.split('-')
df_B['workload id']=df_B['workload id'].str.join("")
df_B['workload id']=df_B['workload id'].astype(int)


In [ ]:
#Finding Unique Workloads
workloads=list(df_B['workload id'].unique())

In [ ]:
## Segregating by workload Ids
for i in workloads:
    temp=pd.DataFrame(columns=df_B.columns)
    for ind,r in df_B.iterrows():
        if int(r['workload id'])==i:
            temp=temp.append(pd.Series(r[df_B.columns],index=df_B.columns ), ignore_index=True)
            
            #print(temp.shape)
    temp.to_pickle(outpath' + '\\workload_'+ str(i)+'.pkl')
    

In [ ]:
#Pruning metrics by dropping all other metrics
pruned=['latency','driver.BlockManager.memory.onHeapMemUsed_MB.avg',
       'worker_1.Disk_transfers_per_second.dm-0',
       'driver.BlockManager.memory.maxMem_MB.avg',
       'driver.jvm.pools.Code-Cache.committed.avg_period',
       'driver.jvm.pools.Code-Cache.committed.avg_increase',
       'worker_1.Paging_and_Virtual_Memory.pgpgout',
       'executor.jvm.pools.PS-Old-Gen.committed.avg',
       'worker_1.Disk_Read_KB/s.sda',
       'driver.LiveListenerBus.queue.executorManagement.size.avg',
       'worker_2.Paging_and_Virtual_Memory.pgscan_direct_normal']

df=pd.DataFrame()
files=loadWorkloadFileNames(outpath)
for filename in files:
    if filename.endswith(".pkl"):
        df=pd.read_pickle(outpath+filename)
        
        knobs=list(df.columns[0:13])
        for i in df.columns:
            if i not in pruned and i not in knobs:
                df.drop(i,axis=1,inplace=True)
#         print(df.shape)
#         print(df.latency)
        df.to_pickle(outpath+filename)
        df.to_csv(outpath++filename+'.csv',header=True,index=False)
print(df.columns)

In [ ]:
#Dividing the file in WorkloadB_test file and WorkloadB files
files=loadWorkloadFileNames(outpath+'\\WorkloadB\\')
final=pd.DataFrame()
for filename in files:
    if filename.endswith(".pkl"):
        df=pd.read_pickle(outpath'+filename)
        temp1=df.iloc[0:5]
        print(temp1.shape)
        temp1.to_pickle(outpath+"5points"+filename)
        temp1.to_csv(outpath+"5points"+filename+".csv",header=True,index=False)
        temp2=df.iloc[5:]
        final=pd.concat([final,temp2])
print(final.latency)
print(final.shape)

final.to_pickle(outpath+'Test_workloadB.pkl')
final.to_csv(outpath+'Test_workloadB.csv',header=True,index=False)

        